# Coursera Capstone Project : Battle of the Neighborhoods

## Introduction

### The recent upheaval and political unrest in Hong Kong have provided impetus for Hong Kong Citizens to consider migration to Canada. A Hong Kong family intends to migrate to Toronto, Canada and has approached my consultancy firm to perform analysis on the neighborhoods around City of Toronto.  Objectives and briefs provided include optimum living standards and an environment which caters to quality of life.  The family also intends to run their own retail business/store once they settle-in and has asked for a search into the existing nature of business being conducted around the neighborhoods.

### This project will explore the similarities or dissimilarities between neighborhoods in some of the neighborhood clusters in Scarborough, and determine which neighborhood best fit their objectives for migration.


## Conclusions

### Our recommendations to the Hong Kong family are as follows :
### 1.	Based on the results reflected in the Analysis below, the family should endeavour to reside in Cluster_0 of Scarborough, where amenities are more readily available.  This will allow them to pursue optimum lifestyle.  
### 2.	In terms of retail business/store, based on information collated from “10 Most Common Venues”, there seems to be a lack of pharmacy and medical center around the neighborhoods. This is an area of business which the Hong Kong family may want to consider.

# Segmenting and Clustering Neighborhoods in Toronto using Toronto Neighborhood Wikipedia data

## Import all necessary libraries

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


vinc

## Extract Toronto Dataset

In [2]:
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header = 0)
df_toronto = table[0]
df_toronto.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Transform Dataset

In [3]:
df_toronto.rename(columns = {"Postcode": "PostalCode", "Neighbourhood": "Neighborhood"}, inplace = True)
df_toronto.drop(df_toronto[df_toronto.Borough == 'Not assigned'].index, inplace=True)
df_toronto.loc[85,'Neighborhood'] = 'Queen\'s Park'
print (df_toronto.shape)
df_toronto.head()

(103, 3)


,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
latlong = pd.read_csv("http://cocl.us/Geospatial_data")
latlong.rename(columns = {"Postal code": "PostalCode"}, inplace = True)
latlong.shape
latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Combine neighborhood, latitude and longitude dataframe

In [5]:
neighbor = pd.merge(df_toronto, latlong)
neighbor.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [6]:
print('Toronto has {} boroughs and {} neighborhoods.'.format(
        len(neighbor['Borough'].unique()),
        neighbor.shape[0]
      )
)

Toronto has 10 boroughs and 103 neighborhoods.


## Latitude and longitude values of Toronto, Canada using geopy library

In [7]:
address = "Toronto, CA"

geolocator = Nominatim(user_agent = "foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Toronto, Canada are {}, {}.'.format(latitude,longitude))

The geographical coordinate of Toronto, Canada are 43.6534817, -79.3839347.


## Create Toronto map with Markers on neighborhoods

In [8]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(neighbor['Latitude'], neighbor['Longitude'], neighbor['Borough'], neighbor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
map_toronto       

## To explore Scarborough Neighborhood data

In [9]:
scarborough_data = neighbor[neighbor['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.shape
scarborough_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Find coordinates and create map around Scarborough

In [10]:
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent = 'foursqaure_agent')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Scarborough, CA are {}, {}.'.format(latitude,longitude))

The geographical coordinate of Scarborough, CA are 43.773077, -79.257774.


In [11]:
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color="#3186cc",
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)
map_scarborough

## Utilising Foursquare API to explore Scarborough neighborhood

In [12]:
CLIENT_ID = '3KCJC5JKX1NZWH310HXZI11PIRKMXJX1TC0QXYJJMLFG51LD'
CLIENT_SECRET = '3ZDEBVJSSSMJVLZT2WSVKXYVP4LJ5TMDDQQ5ZWQU43KEUXZS'
VERSION = '20180604'
LIMIT = 30
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: ' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: 3KCJC5JKX1NZWH310HXZI11PIRKMXJX1TC0QXYJJMLFG51LD
CLIENT_SECRET: 3ZDEBVJSSSMJVLZT2WSVKXYVP4LJ5TMDDQQ5ZWQU43KEUXZS


In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [14]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                    latitudes=scarborough_data['Latitude'],
                                    longitudes=scarborough_data['Longitude'])
print(scarborough_venues.shape)
scarborough_venues.head()

(90, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant


In [15]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Birch Cliff, Cliffside West",5,5,5,5,5,5
Cedarbrae,8,8,8,8,8,8
"Clarks Corners, Tam O'Shanter, Sullivan",12,12,12,12,12,12
"Cliffside, Cliffcrest, Scarborough Village West",3,3,3,3,3,3
"Dorset Park, Wexford Heights, Scarborough Town Centre",8,8,8,8,8,8
"Golden Mile, Clairlea, Oakridge",9,9,9,9,9,9
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
"Kennedy Park, Ionview, East Birchmount Park",6,6,6,6,6,6


In [16]:
print('There are {} distinct venues in {} categories.'.format(
    len(scarborough_venues['Venue'].unique()),len(scarborough_venues['Venue Category'].unique())))

There are 82 distinct venues in 56 categories.


## Analysis of Each Neighborhood

In [17]:
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood']
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]
scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Farm,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Gym,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Supermarket,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [18]:
# Mean for ferquency of occurrence of each category
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Farm,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Gym,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Supermarket,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt,0.000000,0.000,0.000000,0.000000,0.0,0.250000,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000,0.25,0.000,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000,0.000,0.000000,0.25,0.00,0.000000,0.000000,0.000000,0.000,0.000
1,"Birch Cliff, Cliffside West",0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.2,0.000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000,0.2,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.000,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000,0.000,0.000000,0.20,0.00,0.000000,0.000000,0.000000,0.000,0.000
2,Cedarbrae,0.000000,0.125,0.125000,0.125000,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.125,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.0,0.000000,0.125000,0.125000,0.0,0.000000,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.000,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000,0.000,0.000000,0.00,0.00,0.000000,0.000000,0.125000,0.000,0.000
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.000000,0.000,0.000000,0.083333,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000,0.083333,0.000000,0.0,0.083333,0.000000,0.000000,0.000,0.0,0.083333,0.083333,0.083333,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.083333,0.083333,0.000000,0.00,0.000,0.00,0.000,0.000000,0.000,0.00,0.000000,0.083333,0.000000,0.000,0.000000,0.166667,0.0,0.000,0.000,0.000000,0.00,0.00,0.000000,0.000000,0.083333,0.000,0.000
4,"Cliffside, Cliffcrest, Scarborough Village West",0.333333,0.000,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.00,0.000,0.00,0.000,0.000000,0.000,0.00,0.333333,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000,0.000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000,0.000
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.125,0.000000,0.000000,0.0,0.000,0.125000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.00,0.125,0.00,0.000,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.0,0.000,0.000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.125,0.125
6,"Golden Mile, Clairlea, Oakridge",0.000000,0.000,0.222222,0.000000,0.0,0.000000,0.000,0.222222,0.111111,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.000,0.111111,0.000,0.00,0.000000,0.000000,0.111111,0.000,0.000000,0.000000,0.0,0.000,0.000,0.000000,0.00,0.00,0.111111,0.000000,0.000000,0.000,0.000
7,"Guildwood, Morningside, West Hill",0.00

## Each neighborhood with the top 5 venues

In [19]:
num_top_venues = 10
for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"-----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp ['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt-----
                       venue  freq
0  Latin American Restaurant  0.25
1             Breakfast Spot  0.25
2                     Lounge  0.25
3               Skating Rink  0.25
4        American Restaurant  0.00
5                  Pet Store  0.00
6          Korean Restaurant  0.00
7         Light Rail Station  0.00
8             Medical Center  0.00
9              Metro Station  0.00


----Birch Cliff, Cliffside West-----
                       venue  freq
0               Skating Rink   0.2
1                       Café   0.2
2      General Entertainment   0.2
3            College Stadium   0.2
4                       Farm   0.2
5        American Restaurant   0.0
6                       Park   0.0
7  Latin American Restaurant   0.0
8         Light Rail Station   0.0
9                     Lounge   0.0


----Cedarbrae-----
                       venue  freq
0                     Bakery  0.12
1                       Bank  0.12
2        Fried Chicken Joint  0.12
3          

## Sorting by Most Common Venues in DataFrame

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 10
Indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indictors[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']
for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind,:], num_top_venues)
neighborhoods_venues_sorted

,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Convenience Store,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
1,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Farm,Café,Vietnamese Restaurant,Gym,Grocery Store,Gas Station,Fried Chicken Joint
2,Cedarbrae,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Department Store,Gym
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Noodle House,Intersection,Chinese Restaurant,Fast Food Restaurant,Italian Restaurant,Fried Chicken Joint,Bank,Gas Station,Thai Restaurant
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Intersection,Motel,Convenience Store,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
5,"Dorset Park, Wexford Heights, Scarborough Town...",Indian Restaurant,Vietnamese Restaurant,Thrift / Vintage Store,Brewery,Light Rail Station,Chinese Restaurant,Pet Store,Department Store,Grocery Store,General Entertainment
6,"Golden Mile, Clairlea, Oakridge",Bus Line,Bakery,Ice Cream Shop,Metro Station,Bus Station,Park,Soccer Field,Bar,Department Store,Grocery Store
7,"Guildwood, Morningside, West Hill",Restaurant,Breakfast Spot,Medical Center,Electronics Store,Rental Car Location,Mexican Restaurant,Intersection,Bank,Fried Chicken Joint,Convenience Store
8,"Kennedy Park, Ionview, East Birchmount Park",Hobby Shop,Department Store,Bus Station,Chinese Restaurant,Coffee Shop,Discount Store,Hakka Restaurant,Gym,Grocery Store,General Entertainment
9,"Malvern, Rouge",Fast Food Restaurant,Vietnamese Restaurant,Thrift / Vintage Store,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Farm


In [22]:
neighborhoods_venues_sorted.iloc[11,]

Neighborhood              Rouge Hill, Port Union, Highland Creek
1th Most Common Venue                                        Bar
2th Most Common Venue                      Vietnamese Restaurant
3th Most Common Venue                          Convenience Store
4th Most Common Venue                           Hakka Restaurant
5th Most Common Venue                                        Gym
6th Most Common Venue                              Grocery Store
7th Most Common Venue                      General Entertainment
8th Most Common Venue                                Gas Station
9th Most Common Venue                        Fried Chicken Joint
10th Most Common Venue                      Fast Food Restaurant
Name: 11, dtype: object

## Cluster the Scarborough Neighborhood using k-means

In [23]:
# Set number of clusters
kclusters = 3
scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood',1)
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(scarborough_grouped_clustering)
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 0, 0, 0], dtype=int32)

In [24]:
scarborough_data.drop(scarborough_data[scarborough_data.Neighborhood == 'Upper Rouge'].index, inplace = True)
scarborough_merged = scarborough_data
scarborough_merged['Cluster Labels'] = kmeans.labels_
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
scarborough_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0,Fast Food Restaurant,Vietnamese Restaurant,Thrift / Vintage Store,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Farm
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,Bar,Vietnamese Restaurant,Convenience Store,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Restaurant,Breakfast Spot,Medical Center,Electronics Store,Rental Car Location,Mexican Restaurant,Intersection,Bank,Fried Chicken Joint,Convenience Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Convenience Store,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Department Store,Gym


In [25]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start = 11)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = {}
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + 'Cluster' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

## Examine clusters, define categories and assign names for each clusters

In [26]:
scarborough_cluster_0 = scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(4, scarborough_merged.shape[1]))]]
scarborough_cluster_1 = scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(4, scarborough_merged.shape[1]))]]
scarborough_cluster_2 = scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(4, scarborough_merged.shape[1]))]]

In [27]:
scarborough_cluster_0

,Borough,Longitude,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,-79.194353,0,Fast Food Restaurant,Vietnamese Restaurant,Thrift / Vintage Store,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Farm
1,Scarborough,-79.160497,0,Bar,Vietnamese Restaurant,Convenience Store,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,Scarborough,-79.188711,0,Restaurant,Breakfast Spot,Medical Center,Electronics Store,Rental Car Location,Mexican Restaurant,Intersection,Bank,Fried Chicken Joint,Convenience Store
3,Scarborough,-79.216917,0,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Convenience Store,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
4,Scarborough,-79.239476,0,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Department Store,Gym
5,Scarborough,-79.239476,0,Playground,Vietnamese Restaurant,College Stadium,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm
6,Scarborough,-79.262029,0,Hobby Shop,Department Store,Bus Station,Chinese Restaurant,Coffee Shop,Discount Store,Hakka Restaurant,Gym,Grocery Store,General Entertainment
7,Scarborough,-79.284577,0,Bus Line,Bakery,Ice Cream Shop,Metro Station,Bus Station,Park,Soccer Field,Bar,Department Store,Grocery Store
8,Scarborough,-79.239476,0,American Restaurant,Intersection,Motel,Convenience Store,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
9,Scarborough,-79.264848,0,College Stadium,General Entertainment,Skating Rink,Farm,Café,Vietnamese Restaurant,Gym,Grocery Store,Gas Station,Fried Chicken Joint


In [28]:
scarborough_cluster_1

,Borough,Longitude,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Scarborough,-79.273304,1,Indian Restaurant,Vietnamese Restaurant,Thrift / Vintage Store,Brewery,Light Rail Station,Chinese Restaurant,Pet Store,Department Store,Grocery Store,General Entertainment
12,Scarborough,-79.262029,1,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Convenience Store,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint


In [29]:
scarborough_cluster_2

,Borough,Longitude,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Scarborough,-79.295849,2,Bakery,Smoke Shop,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,Convenience Store,Gym,Grocery Store,General Entertainment,Gas Station
